# Operaciones _matriciales_ en TensorFlow

In [1]:
import tensorflow as tf
import numpy as np
# Crear un grafo computacional para esta sesión
session = tf.Session()

## Creación de matrices

In [2]:
# Matriz identidad, una de las tantas formas de realizarlo
identity = tf.diag(tf.ones(4))
session.run(identity)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [3]:
# Una matriz llena de valores aleatorios
A = tf.truncated_normal((3, 4))
print(session.run(A))

[[ 0.99238974  0.32170877  1.5484488   0.84649956]
 [ 1.1694679   0.20511162  1.0814606   1.8884525 ]
 [-0.01525155  1.8661995  -1.5225859   0.5401231 ]]


In [4]:
# Matriz llena de un solo valor
B = tf.fill((3, 4), 1988.)
print(session.run(B))

[[1988. 1988. 1988. 1988.]
 [1988. 1988. 1988. 1988.]
 [1988. 1988. 1988. 1988.]]


In [5]:
# Matrices más grandes, más valores aleatorios
C = tf.random_uniform((4, 3))
print(session.run(C))

[[0.4796245  0.75492287 0.0679332 ]
 [0.7886263  0.93251014 0.8967694 ]
 [0.5253099  0.09135985 0.7080579 ]
 [0.36443138 0.5203539  0.9372779 ]]


In [6]:
# También se puede hacer lo mismo con otras estructuras de datos, por ejemplo numpy
data = np.array([[1.,2.,3.],[4.,5.,6.],[7.,8.,9.]])
D = tf.convert_to_tensor(data)
print(session.run(D))

[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


## Operaciones básicas con matrices

In [7]:
# Suma/resta de matrices
print(session.run(A + B))
print(session.run(B - B))

[[1987.8171 1989.7739 1989.033  1987.5425]
 [1987.1799 1987.7605 1987.8306 1989.6538]
 [1987.7167 1987.754  1987.7773 1988.2693]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [8]:
# Multiplicación
print(session.run(tf.matmul(A, C)))

[[-0.23332632  0.01048076  0.21853335]
 [-1.3401314  -0.74868375 -1.6353986 ]
 [-2.8964427  -3.0990481  -1.9950173 ]]


## Operaciones convencionales del álgebra lineal

In [9]:
# Transpuesta de una matriz
print(session.run(tf.transpose(D)))

[[1. 4. 7.]
 [2. 5. 8.]
 [3. 6. 9.]]


In [10]:
# Determinante de una matriz
print(session.run(tf.matrix_determinant(D)))

6.66133814775094e-16


### NOTA
Esta _nota_ es más bien matemática: con la operación anterior se puede dar cuenta fácilmente que el determinante de la matriz D es cero, por lo tanto su inversa no debe existir.

In [11]:
# Comprobar que la inversa de D no existe, debe mostrar algún error
print(session.run(tf.matrix_inverse(D)))

[[-4.50359963e+15  9.00719925e+15 -4.50359963e+15]
 [ 9.00719925e+15 -1.80143985e+16  9.00719925e+15]
 [-4.50359963e+15  9.00719925e+15 -4.50359963e+15]]


Los valores son valores muy grandes, no tienen sentido; no se mostró un error pero los valores obtenidos no sirven de mucho. Sin embargo, existe la [pseudoinversión](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse) que es la inversa de una matriz para matrices con determinante igual a cero.

In [12]:
# Para que esto funcione se debe tener instalado tensorflow_probability
# Si no se tiene, la siguiente linea de numpy se debe activar
# from numpy.linalg import pinv
import tensorflow_probability as tfp
# Calcular la pseudoinversa de D
print(session.run(tfp.math.pinv(D)))
# Para cuando no se tiene tensorflow_probability, se emplea la implementación de numpy
# print(pinv(D))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

[[-6.38888889e-01 -1.66666667e-01  3.05555556e-01]
 [-5.55555556e-02 -3.12250226e-17  5.55555556e-02]
 [ 5.27777778e-01  1.66666667e-01 -1.94444444e-01]]


Se encuentra ahora que la matriz existe y tiene valores útiles. Esta operación matricial, muy desconocida, tiene aplicaciones cuando se diseñan algoritmos de regresión mediante mínimos cuadrados. También en clasificación al aplicar un método de _Gaussian Naive Bayes_, entre muchas otras aplicaciones muy importantes.

In [13]:
# Matriz que tiene determinante diferente de cero
E = tf.convert_to_tensor(np.array([[1.,2.,3.], [3., 6., -1.], [0., 5., -3.]], dtype=np.float32))
print(session.run(tf.matrix_determinant(E)))

# Por lo tanto la matriz debe tener inversa, y su pseudoinversa debe ser casi igual dentro de un margen de error numérico
print('Inversión convencional, por descomposición de Cholesky')
print(session.run(tf.matrix_inverse(E)))
print('Inversión de Moore-Penrose')
print(session.run(tfp.math.pinv(E)))

50.000004
Inversión convencional, por descomposición de Cholesky
[[-0.26000002  0.42000002 -0.40000004]
 [ 0.18       -0.06        0.2       ]
 [ 0.3        -0.10000001  0.        ]]
Inversión de Moore-Penrose
[[-2.60000020e-01  4.20000017e-01 -4.00000036e-01]
 [ 1.80000007e-01 -5.99999949e-02  2.00000003e-01]
 [ 2.99999982e-01 -1.00000024e-01 -8.94069672e-08]]


## Descomposición de matrices

Manejar las matrices como son no siempre es conveniente, y por lo tanto se crearon las descomposiciones matriciales como LU, QR o Cholesky. A continuación se estudia un ejemplo muy sencillo con la descomposición de Cholesky.

Para encontrar la inversa se debe resolver el sistema de ecuaciones $\mathbf{A} \mathbf{x} = \mathbf{b}$ donde $\mathbf{b}$ es un vector unitario de la matriz identidad.

In [17]:
# Se pretende encontrar la inversa de E, dado que se conoce del paso anterior
# Primero, descomponer la matriz
print(session.run(tf.linalg.qr(E)))
q, r = tf.linalg.qr(E)
# Se calcula y
y_1 = tf.linalg.matvec(tf.transpose(q), [1.0, 0.0, 0.0])
# Reacomodar el tensor
y_1 = tf.reshape(y_1, [3, 1])
# Resolver el sistema de ecuaciones
x_1 = tf.linalg.solve(r, y_1)
# Esta es la primera columna de la matriz E, ver arriba
print(session.run(x_1))

# Aplicar lo mismo para los demás vectores
y_2 = tf.linalg.matvec(tf.transpose(q), [0.0, 1.0, 0.0])
y_2 = tf.reshape(y_2, [3, 1])
x_2 = tf.linalg.solve(r, y_2)
y_3 = tf.linalg.matvec(tf.transpose(q), [0.0, 0.0, 1.0])
y_3 = tf.reshape(y_3, [3, 1])
x_3 = tf.linalg.solve(r, y_3)

# Juntar los resultados en una matriz, reajustando los tamaños de los tensores
x_res = tf.stack([tf.reshape(x_1, [-1]), tf.reshape(x_2,[-1]), tf.reshape(x_3, [-1])], axis=1)
# Mostrar el resultado final como una matriz
# Este es el mismo resultado que antes
print(session.run(x_res))

Qr(q=array([[-0.3162278 ,  0.        ,  0.9486833 ],
       [-0.9486833 ,  0.        , -0.31622773],
       [ 0.        , -1.        ,  0.        ]], dtype=float32), r=array([[-3.1622777e+00, -6.3245554e+00, -2.3841858e-07],
       [ 0.0000000e+00, -5.0000000e+00,  3.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  3.1622777e+00]], dtype=float32))
[[-0.26000002]
 [ 0.18      ]
 [ 0.3       ]]
[[-0.26000002  0.42       -0.4       ]
 [ 0.18       -0.05999999  0.2       ]
 [ 0.3        -0.09999999  0.        ]]


## Vectores y valores propios

In [19]:
# Valores propios
print(session.run(tf.linalg.eigvalsh(E)))

[-5.4780183   0.31889167  9.159128  ]


In [20]:
# Vectores propios
print(session.run(tf.linalg.eigh(E)))

(array([-5.4780183 ,  0.31889167,  9.159128  ], dtype=float32), array([[ 0.20143086, -0.9250799 ,  0.3219515 ],
       [-0.4349576 ,  0.21002677,  0.8756144 ],
       [ 0.8776317 ,  0.31641096,  0.36006463]], dtype=float32))
